In [1]:
!pip install vaderSentiment

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
from nltk.corpus import stopwords


In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [6]:
data = pd.read_csv('/content/News_sentiment_Jan2017_to_Apr2021.csv')

In [7]:
data.head()

,Date,Title,URL,sentiment,confidence,Unnamed: 5
0,05/01/17,Eliminating shadow economy to have positive im...,http://economictimes.indiatimes.com/news/econo...,POSITIVE,0.996185,NaN
1,05/01/17,Two Chinese companies hit roadblock with India...,http://economictimes.indiatimes.com/news/econo...,NEGATIVE,-0.955493,NaN
2,05/01/17,SoftBank India Vision gets new $100,http://economictimes.indiatimes.com/small-biz/...,POSITIVE,0.595612,NaN
3,05/01/17,Nissan halts joint development of luxury cars ...,http://economictimes.indiatimes.com/news/inter...,NEGATIVE,-0.996672,NaN
4,05/01/17,Despite challenges Rajasthan continues to prog...,http://economictimes.indiatimes.com/news/polit...,POSITIVE,0.997388,NaN


In [8]:
data=data[['Date','Title','sentiment']]

In [9]:
pd.set_option('display.max_colwidth',None)

In [10]:
data.head()

,Date,Title,sentiment
0,05/01/17,Eliminating shadow economy to have positive impact on GDP : Arun Jaitley,POSITIVE
1,05/01/17,Two Chinese companies hit roadblock with Indian investments,NEGATIVE
2,05/01/17,SoftBank India Vision gets new $100,POSITIVE
3,05/01/17,Nissan halts joint development of luxury cars with Daimler : Sources,NEGATIVE
4,05/01/17,Despite challenges Rajasthan continues to progress : Vasundhara Raje,POSITIVE


In [11]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    return text


In [13]:
# Apply preprocessing to 'Title' column
data['Title'] = data['Title'].apply(preprocess_text)

In [14]:
data['Title'].isnull().sum()

0

In [15]:
data.head()

,Date,Title,sentiment
0,05/01/17,eliminating shadow economy to have positive impact on gdp arun jaitley,POSITIVE
1,05/01/17,two chinese companies hit roadblock with indian investments,NEGATIVE
2,05/01/17,softbank india vision gets new 100,POSITIVE
3,05/01/17,nissan halts joint development of luxury cars with daimler sources,NEGATIVE
4,05/01/17,despite challenges rajasthan continues to progress vasundhara raje,POSITIVE


In [16]:
analyzer = SentimentIntensityAnalyzer()

In [17]:
#function to get sentiment scores
def get_sentiment_scores(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']


In [18]:
data['SentimentScore'] = data['Title'].apply(get_sentiment_scores)

In [19]:
data.head()

,Date,Title,sentiment,SentimentScore
0,05/01/17,eliminating shadow economy to have positive impact on gdp arun jaitley,POSITIVE,0.5574
1,05/01/17,two chinese companies hit roadblock with indian investments,NEGATIVE,0.0000
2,05/01/17,softbank india vision gets new 100,POSITIVE,0.2500
3,05/01/17,nissan halts joint development of luxury cars with daimler sources,NEGATIVE,0.0000
4,05/01/17,despite challenges rajasthan continues to progress vasundhara raje,POSITIVE,0.3773


In [22]:
data['predicted_sentiment'] = data['SentimentScore'].apply(lambda score: 'POSITIVE' if score >0 else 'NEGATIVE')

In [23]:
data.head()

,Date,Title,sentiment,SentimentScore,predicted_sentiment
0,05/01/17,eliminating shadow economy to have positive impact on gdp arun jaitley,POSITIVE,0.5574,POSITIVE
1,05/01/17,two chinese companies hit roadblock with indian investments,NEGATIVE,0.0000,NEGATIVE
2,05/01/17,softbank india vision gets new 100,POSITIVE,0.2500,POSITIVE
3,05/01/17,nissan halts joint development of luxury cars with daimler sources,NEGATIVE,0.0000,NEGATIVE
4,05/01/17,despite challenges rajasthan continues to progress vasundhara raje,POSITIVE,0.3773,POSITIVE


In [24]:
data.dtypes

Date                    object
Title                   object
sentiment               object
SentimentScore         float64
predicted_sentiment     object
dtype: object

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
data['predicted_sentiment'].value_counts()

NEGATIVE    124710
POSITIVE     75790
Name: predicted_sentiment, dtype: int64

In [27]:
data['sentiment'].value_counts()

NEGATIVE    108118
POSITIVE     92382
Name: sentiment, dtype: int64

In [28]:
accuracy = accuracy_score(data['sentiment'], data['predicted_sentiment'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 60.95%


In [31]:
threshold_values = [0.1,0.2,0.4, 0.5, 0.6]  # Experiment with different threshold values
best_threshold = None
best_accuracy = 0.0

for threshold in threshold_values:
    data['predicted_sentiment'] = data['SentimentScore'].apply(lambda x: 'POSITIVE' if x > threshold else 'NEGATIVE')
    accuracy = accuracy_score(data['sentiment'], data['predicted_sentiment'])

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

print(f"Best Threshold: {best_threshold}, Best Accuracy: {best_accuracy * 100:.2f}%")

Best Threshold: 0.2, Best Accuracy: 61.72%
